In [19]:
from huggingface_hub import login
login()

In [20]:
import pandas as pd
import numpy as np


In [21]:
'''
df_train = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv", index_col=0)
df_test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv", index_col=0)
'''

'\ndf_train = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv", index_col=0)\ndf_test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv", index_col=0)\n'

In [22]:
df_train = pd.read_csv("/media/yanncauchepin/ExternalDisk/Datasets/NaturalLanguageProcessing/disaster_tweets/train.csv", index_col=0)
df_test = pd.read_csv("/media/yanncauchepin/ExternalDisk/Datasets/NaturalLanguageProcessing/disaster_tweets/test.csv", index_col=0)

In [23]:
print(f"Length - train {len(df_train)} - test {len(df_test)}")

Length - train 7613 - test 3263


In [24]:
keywords = pd.concat([
    pd.DataFrame(df_train["keyword"].value_counts()).rename(columns={"count":"train"}),
    pd.DataFrame(df_test["keyword"].value_counts()).rename(columns={"count":"test"})
    ],
    axis=1
)
keywords.head()

,train,test
keyword,,
fatalities,45,5
deluge,42,8
armageddon,42,8
sinking,41,9
damage,41,9


In [25]:
locations = pd.concat([
    pd.DataFrame(df_train["location"].value_counts()).rename(columns={"count":"train"}),
    pd.DataFrame(df_test["location"].value_counts()).rename(columns={"count":"test"})
    ],
    axis=1
)
locations.head()

,train,test
location,,
USA,104.0,37.0
New York,71.0,38.0
United States,50.0,15.0
London,45.0,13.0
Canada,29.0,13.0


In [26]:
df_train_full = df_train.copy()
df_train_full['text'] = df_train_full.apply(lambda row: f"{row['location'] or ''} {row['keyword'] or ''} {row['text']}".strip(), axis=1)
df_test_full = df_test.copy()
df_test_full['text'] = df_test_full.apply(lambda row: f"{row['location'] or ''} {row['keyword'] or ''} {row['text']}".strip(), axis=1)


In [27]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words='english')
train_texts_vec = vectorizer.fit_transform(df_train['text'])
train_texts_vec.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [28]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer()
train_texts_tfidf = tfidf.fit_transform(train_texts_vec)
train_texts_tfidf = train_texts_tfidf.todense()

In [29]:
test_texts_vec = vectorizer.transform(df_test['text'])
test_texts_vec.todense()
test_texts_tfdif = tfidf.transform(test_texts_vec)
test_texts_tfdif = test_texts_tfdif.todense()
X_test = np.asarray(test_texts_tfdif)

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train_texts_tfidf, df_train['target'], test_size = 0.2, stratify=df_train['target'], random_state = 0)

In [31]:
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
X_valid = np.asarray(X_valid)
y_valid = np.asarray(y_valid)

In [32]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer_full = CountVectorizer(stop_words='english')
train_full_texts_vec = vectorizer_full.fit_transform(df_train_full['text'])
train_full_texts_vec.todense()

from sklearn.feature_extraction.text import TfidfTransformer
tfidf_full = TfidfTransformer()
train_full_texts_tfidf = tfidf_full.fit_transform(train_full_texts_vec)
train_full_texts_tfidf = train_full_texts_tfidf.todense()

X_full_train = np.asarray(train_full_texts_tfidf)
y_full_train = np.asarray(df_train_full['target'])

test_full_texts_vec = vectorizer_full.transform(df_test_full['text'])
test_full_texts_vec.todense()
test_full_texts_tfdif = tfidf_full.transform(test_full_texts_vec)
test_full_texts_tfdif = test_full_texts_tfdif.todense()
X_full_test = np.asarray(test_full_texts_tfdif)

In [15]:
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

def evaluate_classifier(y_true, y_pred):
    f1 = f1_score(y_true, y_pred, average='weighted')
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    cm = confusion_matrix(y_true, y_pred)
    
    metrics_df = pd.DataFrame({
        'Metric': ['F1 Score', 'Precision', 'Recall'],
        'Value': [f1, precision, recall]
    })
    
    cm_df = pd.DataFrame(cm, columns=['Predicted Negative', 'Predicted Positive'], index=['Actual Negative', 'Actual Positive'])
    
    return metrics_df, cm_df

In [16]:
from sklearn.naive_bayes import MultinomialNB
naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(X_train, y_train)
y_pred = naive_bayes_classifier.predict(X_valid)

naive_bayes_classifier_assessement = evaluate_classifier(y_valid, y_pred)
print(naive_bayes_classifier_assessement)

(      Metric     Value
0   F1 Score  0.796308
1  Precision  0.807306
2     Recall  0.801051,                  Predicted Negative  Predicted Positive
Actual Negative                 790                  79
Actual Positive                 224                 430)


In [17]:
naive_bayes_classifier_full = MultinomialNB()
naive_bayes_classifier_full.fit(X_full_train, y_full_train)
y_full_pred = naive_bayes_classifier_full.predict(X_full_test)

naive_bayes_classifier_full_submission = pd.DataFrame({
    'id': df_test_full.index,
    'target': y_full_pred
})

In [18]:
'''
from sklearn.svm import SVC
svc_classifier = SVC(kernel='linear')
svc_classifier.fit(X_train, y_train)
y_pred = svc_classifier.predict(X_valid)

svc_classifier_assessement = evaluate_classifier(y_valid, y_pred)
print(svc_classifier_assessement)
'''

"\nfrom sklearn.svm import SVC\nsvc_classifier = SVC(kernel='linear')\nsvc_classifier.fit(X_train, y_train)\ny_pred = svc_classifier.predict(X_valid)\n\nsvc_classifier_assessement = evaluate_classifier(y_valid, y_pred)\nprint(svc_classifier_assessement)\n"

In [19]:
from sklearn.linear_model import LogisticRegression
logistic_regression_classifier = LogisticRegression()
logistic_regression_classifier.fit(X_train, y_train)
y_pred = logistic_regression_classifier.predict(X_valid)

logistic_regression_classifier_assessement = evaluate_classifier(y_valid, y_pred)
print(logistic_regression_classifier_assessement)

(      Metric     Value
0   F1 Score  0.794483
1  Precision  0.810064
2     Recall  0.800394,                  Predicted Negative  Predicted Positive
Actual Negative                 801                  68
Actual Positive                 236                 418)


In [21]:
logistic_regression_classifier_full = LogisticRegression()
logistic_regression_classifier_full.fit(X_full_train, y_full_train)
y_full_pred = logistic_regression_classifier_full.predict(X_full_test)

logistic_regression_classifier_full_submission = pd.DataFrame({
    'id': df_test_full.index,
    'target': y_full_pred
})

In [ ]:
from sklearn.ensemble import RandomForestClassifier
random_forest_classifier = RandomForestClassifier()
random_forest_classifier.fit(X_train, y_train)
y_pred = random_forest_classifier.predict(X_valid)

random_forest_classifier_assessement = evaluate_classifier(y_valid, y_pred)
print(random_forest_classifier_assessement)

(      Metric     Value
0   F1 Score  0.775101
1  Precision  0.802528
2     Recall  0.784636,                  Predicted Negative  Predicted Positive
Actual Negative                 814                  55
Actual Positive                 273                 381)


In [ ]:
random_forest_classifier_full = RandomForestClassifier()
random_forest_classifier_full.fit(X_full_train, y_full_train)
y_full_pred = random_forest_classifier_full.predict(X_full_test)

random_forest_classifier_full_submission = pd.DataFrame({
    'id': df_test_full.index,
    'target': y_full_pred
})

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

neural_network_classifier = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

neural_network_classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history = neural_network_classifier.fit(
    X_train, y_train,
    epochs=10,
    batch_size=8,
    validation_data=(X_valid, y_valid),
    callbacks=[early_stopping]
)

y_pred = neural_network_classifier.predict(X_valid)

neural_network_classifier_assessment = evaluate_classifier(y_valid, np.round(y_pred))
print(neural_network_classifier_assessment)

/home/yanncauchepin/Git/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-10-31 20:49:37.365278: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2024-10-31 20:49:37.365795: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:135] retrieving CUDA diagnostic information for host: yanncauchepincomputing
2024-10-31 20:49:37.365813: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:142] hostname: yanncauchepincomputing
2024-10-31 20:49:37.366105: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:166] libcuda reported version is: 550

Epoch 1/10


2024-10-31 20:49:40.182417: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 520402680 exceeds 10% of free system memory.


762/762 ━━━━━━━━━━━━━━━━━━━━ 14s 16ms/step - accuracy: 0.6765 - loss: 0.5968 - val_accuracy: 0.7984 - val_loss: 0.4524
Epoch 2/10
762/762 ━━━━━━━━━━━━━━━━━━━━ 12s 15ms/step - accuracy: 0.8905 - loss: 0.2714 - val_accuracy: 0.7682 - val_loss: 0.5765
Epoch 3/10
762/762 ━━━━━━━━━━━━━━━━━━━━ 13s 17ms/step - accuracy: 0.9630 - loss: 0.0922 - val_accuracy: 0.7630 - val_loss: 0.7429
Epoch 4/10
762/762 ━━━━━━━━━━━━━━━━━━━━ 12s 15ms/step - accuracy: 0.9837 - loss: 0.0401 - val_accuracy: 0.7525 - val_loss: 0.8938
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
(      Metric     Value
0   F1 Score  0.792614
1  Precision  0.807421
2     Recall  0.798424,                  Predicted Negative  Predicted Positive
Actual Negative                 798                  71
Actual Positive                 236                 418)


In [ ]:
X_full_train_, X_full_valid, y_full_train_, y_full_valid = train_test_split(X_full_train, y_full_train, test_size = 0.15, stratify=y_full_train, random_state = 0)

neural_network_classifier_full = Sequential([
    Dense(64, activation='relu', input_shape=(X_full_train_.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

neural_network_classifier_full.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
early_stopping_full = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history_full = neural_network_classifier_full.fit(
    X_full_train_, y_full_train_,
    epochs=20,
    batch_size=8,
    validation_data=(X_full_valid, y_full_valid),
    callbacks=[early_stopping]
)

y_full_pred = neural_network_classifier_full.predict(X_full_test)

neural_network_classifier_full_submission = pd.DataFrame({
    'id': df_test_full.index,
    'target': np.round(y_full_pred).flatten()
})

/home/yanncauchepin/Git/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
809/809 ━━━━━━━━━━━━━━━━━━━━ 17s 19ms/step - accuracy: 0.6790 - loss: 0.5842 - val_accuracy: 0.8082 - val_loss: 0.4394
Epoch 2/20
809/809 ━━━━━━━━━━━━━━━━━━━━ 14s 17ms/step - accuracy: 0.8946 - loss: 0.2580 - val_accuracy: 0.7723 - val_loss: 0.5667
Epoch 3/20
809/809 ━━━━━━━━━━━━━━━━━━━━ 15s 19ms/step - accuracy: 0.9689 - loss: 0.0890 - val_accuracy: 0.7627 - val_loss: 0.7224
Epoch 4/20
809/809 ━━━━━━━━━━━━━━━━━━━━ 15s 19ms/step - accuracy: 0.9848 - loss: 0.0359 - val_accuracy: 0.7434 - val_loss: 0.9126
102/102 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


In [ ]:
# pip install xgboost catboost

In [ ]:
from xgboost import XGBClassifier

xgb_classifier = XGBClassifier(n_estimators=1000, max_depth=4, learning_rate=0.1, use_label_encoder=False, eval_metric='logloss')
xgb_classifier.fit(X_train, y_train, verbose=1)

y_pred = xgb_classifier.predict(X_valid)

xgb_classifier_assessement = evaluate_classifier(y_valid, y_pred)
print(xgb_classifier_assessement)

/home/yanncauchepin/Git/.venv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [20:58:09] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


(      Metric     Value
0   F1 Score  0.785225
1  Precision  0.794514
2     Recall  0.789888,                  Predicted Negative  Predicted Positive
Actual Negative                 778                  91
Actual Positive                 229                 425)


In [ ]:
xgb_classifier_full = XGBClassifier(n_estimators=1000, max_depth=4, learning_rate=0.1, use_label_encoder=False, eval_metric='logloss')
xgb_classifier_full.fit(X_full_train, y_full_train, verbose=1)

y_full_pred = xgb_classifier_full.predict(X_full_test)

xgb_classifier_full_submission = pd.DataFrame({
    'id': df_test_full.index,
    'target': y_full_pred
})

/home/yanncauchepin/Git/.venv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [21:00:38] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [ ]:
from catboost import CatBoostClassifier

catboost_classifier = CatBoostClassifier(iterations=1000, depth=4, learning_rate=0.1, verbose=2)
catboost_classifier.fit(X_train, y_train)

y_pred = catboost_classifier.predict(X_valid)

catboost_classifier_assessement = evaluate_classifier(y_valid, y_pred)
print(catboost_classifier_assessement)

0:	learn: 0.6797045	total: 74.8ms	remaining: 1m 14s
2:	learn: 0.6640753	total: 127ms	remaining: 42.2s
4:	learn: 0.6527238	total: 177ms	remaining: 35.3s
6:	learn: 0.6418404	total: 225ms	remaining: 31.9s
8:	learn: 0.6339999	total: 272ms	remaining: 30s
10:	learn: 0.6280628	total: 324ms	remaining: 29.1s
12:	learn: 0.6209378	total: 389ms	remaining: 29.5s
14:	learn: 0.6162400	total: 437ms	remaining: 28.7s
16:	learn: 0.6115203	total: 486ms	remaining: 28.1s
18:	learn: 0.6076645	total: 538ms	remaining: 27.8s
20:	learn: 0.6030091	total: 586ms	remaining: 27.3s
22:	learn: 0.5995534	total: 636ms	remaining: 27s
24:	learn: 0.5964546	total: 683ms	remaining: 26.7s
26:	learn: 0.5930054	total: 732ms	remaining: 26.4s
28:	learn: 0.5902966	total: 781ms	remaining: 26.2s
30:	learn: 0.5873470	total: 828ms	remaining: 25.9s
32:	learn: 0.5849357	total: 877ms	remaining: 25.7s
34:	learn: 0.5827788	total: 926ms	remaining: 25.5s
36:	learn: 0.5798956	total: 975ms	remaining: 25.4s
38:	learn: 0.5778346	total: 1.02s	rema

In [ ]:
catboost_classifier_full = CatBoostClassifier(iterations=1000, depth=4, learning_rate=0.1, verbose=2)
catboost_classifier_full.fit(X_full_train, y_full_train)

y_full_pred = catboost_classifier_full.predict(X_full_test)

catboost_classifier_full_submission = pd.DataFrame({
    'id': df_test_full.index,
    'target': y_full_pred
})

0:	learn: 0.6830062	total: 28.3ms	remaining: 28.3s
2:	learn: 0.6635779	total: 83.1ms	remaining: 27.6s
4:	learn: 0.6506161	total: 136ms	remaining: 27s
6:	learn: 0.6415673	total: 188ms	remaining: 26.7s
8:	learn: 0.6337147	total: 240ms	remaining: 26.5s
10:	learn: 0.6258401	total: 295ms	remaining: 26.6s
12:	learn: 0.6205170	total: 347ms	remaining: 26.3s
14:	learn: 0.6152465	total: 400ms	remaining: 26.3s
16:	learn: 0.6112210	total: 453ms	remaining: 26.2s
18:	learn: 0.6066602	total: 507ms	remaining: 26.2s
20:	learn: 0.6036622	total: 559ms	remaining: 26.1s
22:	learn: 0.6004812	total: 612ms	remaining: 26s
24:	learn: 0.5975389	total: 665ms	remaining: 25.9s
26:	learn: 0.5946601	total: 718ms	remaining: 25.9s
28:	learn: 0.5914057	total: 771ms	remaining: 25.8s
30:	learn: 0.5874459	total: 824ms	remaining: 25.8s
32:	learn: 0.5840149	total: 880ms	remaining: 25.8s
34:	learn: 0.5819688	total: 936ms	remaining: 25.8s
36:	learn: 0.5796179	total: 989ms	remaining: 25.7s
38:	learn: 0.5774566	total: 1.04s	rema

In [ ]:
# pip install transformers torch

In [ ]:
# pip install datasets

In [ ]:
# !pip install hugging_face_hub

In [38]:
import torch
from transformers import Trainer, TrainingArguments
from datasets import Dataset

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

bert_tokenizer_full = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model_full = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

bert_encodings_full = bert_tokenizer_full(list(df_train_full['text']), truncation=True, padding=True, max_length=256)
bert_labels_full = torch.tensor(list(df_train_full['target']))

bert_input_ids_train_full, bert_input_ids_valid_full, bert_token_type_ids_train_full, bert_token_type_ids_valid_full, \
bert_attention_mask_train_full, bert_attention_mask_valid_full, bert_y_train_full, bert_y_valid_full = train_test_split(
    bert_encodings_full['input_ids'], 
    bert_encodings_full['token_type_ids'], 
    bert_encodings_full['attention_mask'], 
    bert_labels_full, 
    test_size=0.15, 
    stratify=bert_labels_full, 
    random_state=0
)

bert_train_encodings_full = {
    'input_ids': torch.tensor(bert_input_ids_train_full),
    'token_type_ids': torch.tensor(bert_token_type_ids_train_full),
    'attention_mask': torch.tensor(bert_attention_mask_train_full)
}

bert_valid_encodings_full = {
    'input_ids': torch.tensor(bert_input_ids_valid_full),
    'token_type_ids': torch.tensor(bert_token_type_ids_valid_full),
    'attention_mask': torch.tensor(bert_attention_mask_valid_full)
}


training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    eval_strategy="epoch",
    report_to="none"
)

bert_train_dataset_full = Dataset.from_dict({
    "input_ids": bert_train_encodings_full['input_ids'],
    "attention_mask": bert_train_encodings_full['attention_mask'],
    "labels": bert_y_train_full
})

bert_valid_dataset_full = Dataset.from_dict({
    "input_ids": bert_valid_encodings_full['input_ids'],
    "attention_mask": bert_valid_encodings_full['attention_mask'],
    "labels": bert_y_valid_full
})

bert_trainer_full = Trainer(
    model=bert_model_full,
    args=training_args,
    train_dataset=bert_train_dataset_full,
    eval_dataset=bert_valid_dataset_full
)

bert_trainer_full.train()

bert_predictions_full = bert_trainer_full.predict(bert_valid_dataset_full)
bert_logits_full = bert_predictions_full.predictions
bert_y_pred_full = np.argmax(bert_logits_full, axis=1)

bert_trainer_full_assessement = evaluate_classifier(bert_y_valid_full.numpy(), bert_y_pred_full)

bert_trainer_full.save_model("disastertweets_bert_model")
bert_trainer_full.push_to_hub("yanncauchepin/kaggle_disastertweets_bert_model")

bert_test_encodings_full = bert_tokenizer_full(list(df_test_full['text']), truncation=True, padding=True, max_length=256)
bert_test_encodings_full = {key: torch.tensor(val) for key, val in bert_test_encodings_full.items()}
bert_test_dataset_full = Dataset.from_dict({
    "input_ids": bert_test_encodings_full['input_ids'],
    "attention_mask": bert_test_encodings_full['attention_mask']
})

bert_test_predictions_full = bert_trainer_full.predict(bert_test_dataset_full)
bert_test_logits_full = bert_test_predictions_full.predictions
bert_test_y_pred_full = np.argmax(bert_test_logits_full, axis=1)

bert_test_submission_full = pd.DataFrame({
    'id': df_test_full.index,
    'target': bert_test_y_pred_full.flatten()
})

hf_bert_test_submission_full = Dataset.from_pandas(bert_test_submission_full)
hf_bert_test_submission_full.push_to_hub("yanncauchepin/kaggle_disastertweets_bert_submission_df")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


'\nbert_labels_full = torch.tensor(list(df_train_full[\'target\']))\n\nbert_input_ids_train_full, bert_input_ids_valid_full, bert_token_type_ids_train_full, bert_token_type_ids_valid_full, bert_attention_mask_train_full, bert_attention_mask_valid_full, bert_y_train_full, bert_y_valid_full = train_test_split(\n    bert_encodings_full[\'input_ids\'], \n    bert_encodings_full[\'token_type_ids\'], \n    bert_encodings_full[\'attention_mask\'], \n    bert_labels_full, \n    test_size=0.15, \n    stratify=bert_labels_full, \n    random_state=0\n)\n\nbert_train_encodings_full = {\n    \'input_ids\': torch.tensor(bert_input_ids_train_full),\n    \'token_type_ids\': torch.tensor(bert_token_type_ids_train_full),\n    \'attention_mask\': torch.tensor(bert_attention_mask_train_full)\n}\n\nbert_valid_encodings_full = {\n    \'input_ids\': torch.tensor(bert_input_ids_valid_full),\n    \'token_type_ids\': torch.tensor(bert_token_type_ids_valid_full),\n    \'attention_mask\': torch.tensor(bert_attent

In [35]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer
import torch
import pandas as pd
from datasets import Dataset
import numpy as np

model = BertForSequenceClassification.from_pretrained("my_model")
trainer = Trainer(model=model)
trainer.push_to_hub("yanncauchepin/kaggle_disastertweets_bert_model")

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.24k [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
# !pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 1.3 MB/s eta 0:00:00a 0:00:01


In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

distill_bert_tokenizer_full = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
distill_bert_model_full = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

distill_bert_encodings_full = distill_bert_tokenizer_full(list(df_train_full['text']), truncation=True, padding=True, max_length=256)
distill_bert_labels_full = torch.tensor(list(df_train_full['target']))

distill_bert_input_ids_train_full, distill_bert_input_ids_valid_full, \
distill_bert_attention_mask_train_full, distill_bert_attention_mask_valid_full, distill_bert_y_train_full, distill_bert_y_valid_full = train_test_split(
    distill_bert_encodings_full['input_ids'], 
    distill_bert_encodings_full['attention_mask'], 
    distill_bert_labels_full, 
    test_size=0.15, 
    stratify=distill_bert_labels_full, 
    random_state=0
)

distill_bert_train_encodings_full = {
    'input_ids': torch.tensor(distill_bert_input_ids_train_full),
    'attention_mask': torch.tensor(distill_bert_attention_mask_train_full)
}

distill_bert_valid_encodings_full = {
    'input_ids': torch.tensor(distill_bert_input_ids_valid_full),
    'attention_mask': torch.tensor(distill_bert_attention_mask_valid_full)
}


training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    eval_strategy="epoch",
    report_to="none"
)

distill_bert_train_dataset_full = Dataset.from_dict({
    "input_ids": distill_bert_train_encodings_full['input_ids'],
    "attention_mask": distill_bert_train_encodings_full['attention_mask'],
    "labels": distill_bert_y_train_full
})

distill_bert_valid_dataset_full = Dataset.from_dict({
    "input_ids": distill_bert_valid_encodings_full['input_ids'],
    "attention_mask": distill_bert_valid_encodings_full['attention_mask'],
    "labels": distill_bert_y_valid_full
})

distill_bert_trainer_full = Trainer(
    model=distill_bert_model_full,
    args=training_args,
    train_dataset=distill_bert_train_dataset_full,
    eval_dataset=distill_bert_valid_dataset_full
)

distill_bert_trainer_full.train()

distill_bert_predictions_full = distill_bert_trainer_full.predict(distill_bert_valid_dataset_full)
distill_bert_logits_full = distill_bert_predictions_full.predictions
distill_bert_y_pred_full = np.argmax(distill_bert_logits_full, axis=1)

distill_bert_trainer_full_assessement = evaluate_classifier(distill_bert_y_valid_full.numpy(), distill_bert_y_pred_full)

distill_bert_test_encodings_full = distill_bert_tokenizer_full(list(df_test_full['text']), truncation=True, padding=True, max_length=256)
distill_bert_test_encodings_full = {key: torch.tensor(val) for key, val in distill_bert_test_encodings_full.items()}
distill_bert_test_dataset_full = Dataset.from_dict({
    "input_ids": distill_bert_test_encodings_full['input_ids'],
    "attention_mask": distill_bert_test_encodings_full['attention_mask']
})

distill_bert_test_predictions_full = distill_bert_trainer_full.predict(distill_bert_test_dataset_full)
distill_bert_test_logits_full = distill_bert_test_predictions_full.predictions
distill_bert_test_y_pred_full = np.argmax(distill_bert_test_logits_full, axis=1)

distill_bert_test_submission_full = pd.DataFrame({
    'id': df_test_full.index,
    'target': distill_bert_test_y_pred_full.flatten()
})

distill_bert_trainer_full.push_to_hub("yanncauchepin/kaggle_disastertweets_distill_bert_model")
distill_bert_tokenizer_full.push_to_hub("yanncauchepin/kaggle_disastertweets_distill_bert_tokenizer")

hf_distill_bert_test_submission_full = Dataset.from_pandas(distill_bert_test_submission_full)
hf_distill_bert_test_submission_full.push_to_hub("yanncauchepin/kaggle_disastertweets_distill_bert_submission_df")

In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification

robert_tokenizer_full = RobertaTokenizer.from_pretrained("roberta-base")
robert_model_full = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)

robert_encodings_full = robert_tokenizer_full(list(df_train_full['text']), truncation=True, padding=True, max_length=256)
robert_labels_full = torch.tensor(list(df_train_full['target']))

robert_input_ids_train_full, robert_input_ids_valid_full, robert_token_type_ids_train_full, robert_token_type_ids_valid_full, \
robert_attention_mask_train_full, robert_attention_mask_valid_full, robert_y_train_full, robert_y_valid_full = train_test_split(
    robert_encodings_full['input_ids'], 
    robert_encodings_full['token_type_ids'], 
    robert_encodings_full['attention_mask'], 
    robert_labels_full, 
    test_size=0.15, 
    stratify=robert_labels_full, 
    random_state=0
)

robert_train_encodings_full = {
    'input_ids': torch.tensor(robert_input_ids_train_full),
    'token_type_ids': torch.tensor(robert_token_type_ids_train_full),
    'attention_mask': torch.tensor(robert_attention_mask_train_full)
}

robert_valid_encodings_full = {
    'input_ids': torch.tensor(robert_input_ids_valid_full),
    'token_type_ids': torch.tensor(robert_token_type_ids_valid_full),
    'attention_mask': torch.tensor(robert_attention_mask_valid_full)
}


training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    eval_strategy="epoch",
    report_to="none"
)

robert_train_dataset_full = Dataset.from_dict({
    "input_ids": robert_train_encodings_full['input_ids'],
    "attention_mask": robert_train_encodings_full['attention_mask'],
    "labels": robert_y_train_full
})

robert_valid_dataset_full = Dataset.from_dict({
    "input_ids": robert_valid_encodings_full['input_ids'],
    "attention_mask": robert_valid_encodings_full['attention_mask'],
    "labels": robert_y_valid_full
})

robert_trainer_full = Trainer(
    model=robert_model_full,
    args=training_args,
    train_dataset=robert_train_dataset_full,
    eval_dataset=robert_valid_dataset_full
)

robert_trainer_full.train()

robert_predictions_full = robert_trainer_full.predict(robert_valid_dataset_full)
robert_logits_full = robert_predictions_full.predictions
robert_y_pred_full = np.argmax(robert_logits_full, axis=1)

robert_trainer_full_assessement = evaluate_classifier(robert_y_valid_full.numpy(), robert_y_pred_full)

robert_test_encodings_full = robert_tokenizer_full(list(df_test_full['text']), truncation=True, padding=True, max_length=256)
robert_test_encodings_full = {key: torch.tensor(val) for key, val in robert_test_encodings_full.items()}
robert_test_dataset_full = Dataset.from_dict({
    "input_ids": robert_test_encodings_full['input_ids'],
    "attention_mask": robert_test_encodings_full['attention_mask']
})

robert_test_predictions_full = robert_trainer_full.predict(robert_test_dataset_full)
robert_test_logits_full = robert_test_predictions_full.predictions
robert_test_y_pred_full = np.argmax(robert_test_logits_full, axis=1)

robert_test_submission_full = pd.DataFrame({
    'id': df_test_full.index,
    'target': robert_test_y_pred_full.flatten()
})

robert_trainer_full.push_to_hub("yanncauchepin/kaggle_disastertweets_robert_model")
robert_tokenizer_full.push_to_hub("yanncauchepin/kaggle_disastertweets_robert_tokenizer")

hf_robert_test_submission_full = Dataset.from_pandas(robert_test_submission_full)
hf_robert_test_submission_full.push_to_hub("yanncauchepin/kaggle_disastertweets_robert_submission_df")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/yanncauchepin/Git/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:716: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
  0%|          | 10/4569 [00:16<1:58:22,  1.56s/it]

{'loss': 0.6974, 'grad_norm': 2.794342517852783, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.01}


  0%|          | 20/4569 [00:32<1:59:26,  1.58s/it]

{'loss': 0.7033, 'grad_norm': 2.9624812602996826, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.01}


  1%|          | 30/4569 [00:48<2:02:36,  1.62s/it]

...

100%|█████████▉| 4560/4569 [2:12:23<00:15,  1.69s/it]

{'loss': 0.1803, 'grad_norm': 1.1258264780044556, 'learning_rate': 1.1059228311624479e-07, 'epoch': 2.99}


100%|██████████| 4569/4569 [2:14:45<00:00,  1.77s/it]


{'eval_loss': 0.6005200743675232, 'eval_runtime': 113.6187, 'eval_samples_per_second': 13.404, 'eval_steps_per_second': 3.353, 'epoch': 3.0}
{'train_runtime': 8085.0579, 'train_samples_per_second': 2.26, 'train_steps_per_second': 0.565, 'train_loss': 0.6373474184912074, 'epoch': 3.0}


100%|██████████| 816/816 [03:53<00:00,  3.49it/s]


In [ ]:
# !pip install sentencepiece

In [ ]:
import sentencepiece
from transformers import AlbertTokenizer, AlbertForSequenceClassification

albert_tokenizer_full = AlbertTokenizer.from_pretrained("albert-base-v2")
albert_model_full = AlbertForSequenceClassification.from_pretrained("albert-base-v2", num_labels=2)

albert_encodings_full = albert_tokenizer_full(list(df_train_full['text']), truncation=True, padding=True, max_length=256)
albert_labels_full = torch.tensor(list(df_train_full['target']))

albert_input_ids_train_full, albert_input_ids_valid_full, albert_token_type_ids_train_full, albert_token_type_ids_valid_full, \
albert_attention_mask_train_full, albert_attention_mask_valid_full, albert_y_train_full, albert_y_valid_full = train_test_split(
    albert_encodings_full['input_ids'], 
    albert_encodings_full['token_type_ids'], 
    albert_encodings_full['attention_mask'], 
    albert_labels_full, 
    test_size=0.15, 
    stratify=albert_labels_full, 
    random_state=0
)

albert_train_encodings_full = {
    'input_ids': torch.tensor(albert_input_ids_train_full),
    'token_type_ids': torch.tensor(albert_token_type_ids_train_full),
    'attention_mask': torch.tensor(albert_attention_mask_train_full)
}

albert_valid_encodings_full = {
    'input_ids': torch.tensor(albert_input_ids_valid_full),
    'token_type_ids': torch.tensor(albert_token_type_ids_valid_full),
    'attention_mask': torch.tensor(albert_attention_mask_valid_full)
}


training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    eval_strategy="epoch",
    report_to="none"
)

albert_train_dataset_full = Dataset.from_dict({
    "input_ids": albert_train_encodings_full['input_ids'],
    "attention_mask": albert_train_encodings_full['attention_mask'],
    "labels": albert_y_train_full
})

albert_valid_dataset_full = Dataset.from_dict({
    "input_ids": albert_valid_encodings_full['input_ids'],
    "attention_mask": albert_valid_encodings_full['attention_mask'],
    "labels": albert_y_valid_full
})

albert_trainer_full = Trainer(
    model=albert_model_full,
    args=training_args,
    train_dataset=albert_train_dataset_full,
    eval_dataset=albert_valid_dataset_full
)

albert_trainer_full.train()

albert_predictions_full = albert_trainer_full.predict(albert_valid_dataset_full)
albert_logits_full = albert_predictions_full.predictions
albert_y_pred_full = np.argmax(albert_logits_full, axis=1)

albert_trainer_full_assessement = evaluate_classifier(albert_y_valid_full.numpy(), albert_y_pred_full)

albert_test_encodings_full = albert_tokenizer_full(list(df_test_full['text']), truncation=True, padding=True, max_length=256)
albert_test_encodings_full = {key: torch.tensor(val) for key, val in albert_test_encodings_full.items()}
albert_test_dataset_full = Dataset.from_dict({
    "input_ids": albert_test_encodings_full['input_ids'],
    "attention_mask": albert_test_encodings_full['attention_mask']
})

albert_test_predictions_full = albert_trainer_full.predict(albert_test_dataset_full)
albert_test_logits_full = albert_test_predictions_full.predictions
albert_test_y_pred_full = np.argmax(albert_test_logits_full, axis=1)

albert_test_submission_full = pd.DataFrame({
    'id': df_test_full.index,
    'target': albert_test_y_pred_full.flatten()
})

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/yanncauchepin/Git/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:716: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
                                                  
  0%|          | 4/4854 [01:36<2:58:15,  2.21s/it] 

{'loss': 0.6855, 'grad_norm': 20.285402297973633, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.01}


                                                  
  0%|          | 4/4854 [01:51<2:58:15,  2.21s/it] 

{'loss': 0.7198, 'grad_norm': 19.609766006469727, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.01}


KeyboardInterrupt: 